 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Theano-Implementation" data-toc-modified-id="Theano-Implementation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Theano Implementation</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Keras-implementation" data-toc-modified-id="Keras-implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Keras implementation</a></span></li></ul></div>

# EMOS Network

In [56]:
# Imports
from importlib import reload
import emos_network_theano; reload(emos_network_theano)
from  emos_network_theano import EMOS_Network
import utils; reload(utils)
from utils import *

In [3]:
# Basic setup
DATA_DIR = '/Users/stephanrasp/repositories/ppnn/data/'  # Mac
fn = 'data_interpolated.nc'
window_size = 50
fclt = 48
utc = 0

## Theano Implementation

In [4]:
model = EMOS_Network()

In [5]:
date_idx = 1507

In [6]:
tobs_full, tfc_full, dates = load_nc_data(DATA_DIR + fn, utc=utc)

In [7]:
tobs_full.shape

(3653, 537)

In [8]:
tfc_mean_train, tfc_std_train, tobs_train, \
    tfc_mean_test, tfc_std_test, tobs_test = \
        get_train_test_data(tobs_full, tfc_full, date_idx, 
                            window_size, fclt)

In [9]:
lr = np.asarray(0.1, dtype='float32')
early_stopping_delta = 0.000001

In [10]:
model.fit(tfc_mean_train, tfc_std_train, tobs_train, 1000, 
          (tfc_mean_test, tfc_std_test, tobs_test), lr=lr, 
          early_stopping_delta=early_stopping_delta)

(array(1.168661053933833), array(1.0255100450378438))

In [11]:
train_crps_list, valid_crps_list = loop_over_days(
    tobs_full, 
    tfc_full, 
    100, 120, 
    window_size=25,
    fclt=48,     
    epochs_max=2000, 
    early_stopping_delta=1e-6, 
    lr=0.1)

100
120
Time: 4.95 s


## Keras implementation

In [57]:
import keras
from keras.layers import Input, Dense
from keras.models import Model
import keras.backend as K

In [60]:
# Using Tensorflow backend is ok because that way I can use the GPU's at work
# Let's start just with a simple linear T bias correction so we don't
# have to worry about the CRPS
mean_in = Input(shape=(1,))
std_in = Input(shape=(1,))
mean_out = Dense(1, activation='linear')(mean_in)
std_out = Dense(1, activation='linear')(std_in)
x = keras.layers.concatenate([mean_out, std_out], axis=1)

In [61]:
model = Model(inputs=[mean_in, std_in], outputs=x)

In [62]:
from emos_network_theano import crps_cost_function

In [63]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_20 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
input_21 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
dense_20 (Dense)                 (None, 1)             2           input_20[0][0]                   
____________________________________________________________________________________________________
dense_21 (Dense)                 (None, 1)             2           input_21[0][0]                   
___________________________________________________________________________________________

In [64]:
from keras.optimizers import Adam, SGD
opt = SGD(lr=0.1)
model.compile(optimizer=opt, loss=crps_cost_function, metrics=[crps_cost_function])

In [65]:
tfc_mean_train, tfc_std_train, tobs_train, \
    tfc_mean_test, tfc_std_test, tobs_test = \
        get_train_test_data(tobs_full, tfc_full, date_idx, 
                            window_size, fclt)
tfc_mean_train.shape

(25190,)

In [95]:
model.fit([tfc_mean_train, tfc_std_train], tobs_train, epochs=1, batch_size=25190,
          validation_data=[[tfc_mean_test, tfc_std_test], tobs_test], verbose=1)

Train on 25190 samples, validate on 506 samples
Epoch 1/1
25190/25190 [==============================] - 0s - loss: 1.1686 - crps_cost_function: 1.1686 - val_loss: 1.0251 - val_crps_cost_function: 1.0251


In [38]:
inp = Input(shape=(2,))
x = Dense(2, activation='linear')(inp)
model = Model(inputs=inp, outputs=x)

In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.compile(optimizer=opt, loss=crps_cost_function, metrics=[crps_cost_function])

In [41]:
in_train = np.column_stack([tfc_mean_train, tfc_std_train])
in_train.shape

(25190, 2)

In [42]:
model.fit(in_train, tobs_train, epochs=1, batch_size=64)

Epoch 1/1


IndexError: index out of bounds
Apply node that caused the error: Subtensor{::, int64}(/dense_9_target, Constant{1})
Toposort index: 0
Inputs types: [TensorType(float32, matrix), Scalar(int64)]
Inputs shapes: [(64, 1), ()]
Inputs strides: [(4, 4), ()]
Inputs values: ['not shown', 1]
Outputs clients: [[Elemwise{Composite{sqrt(sqr(i0))}}(Subtensor{::, int64}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/Users/stephanrasp/anaconda/envs/py36_keras/lib/python3.6/site-packages/ipykernel/zmqshell.py", line 533, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "/Users/stephanrasp/anaconda/envs/py36_keras/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2698, in run_cell
    interactivity=interactivity, compiler=compiler, result=result)
  File "/Users/stephanrasp/anaconda/envs/py36_keras/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2808, in run_ast_nodes
  